In [1]:
import sys
sys.path.insert(0, '../../src/')

import pandas as pd
import kaggle
import auto_co2 as co2
import importlib
importlib.reload(co2)

2024-01-15 18:56:51.446901: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-15 18:56:51.464139: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


<module 'auto_co2' from '/home/mln/GIT/car-co2-prediction_pre-release/notebooks/../src/auto_co2/__init__.py'>

In [2]:
#auth_file_path = ''
#df = co2.data.download_and_load_co2_data()
df = co2.data.load_co2_data()
df = co2.data.dataviz_preprocess(df)
df['ElectricRange'] = df['ElectricRange'].fillna(0)
df['InnovativeEmissionsReductionWltp'] = df['InnovativeEmissionsReductionWltp'].fillna(0)
df['ElectricConsumption'] = df['ElectricConsumption'].fillna(0)
df = co2.data.standardize_innovtech(df)
df = df.drop(columns=['ID', 'Country', 'ManufacturerName', 'WltpTestMass', 'FuelConsumption', 'RegistrationDate', 'AxleWidthSteering', 'ManufNameOem', 'FuelMode', 'EngineCapacity', 'AxleWidthSteering'])


co2.styles.displayer(df, title='APERCU DU JEU DE DONNEES', save=True)
co2.styles.display_info(df, title='DONNEES', save=True) 
co2.styles.display_na(df, title='PART DE VALEURS MANQUANTES DANS LE JEU DE DONNEES', save=True)   


Incomplete rows dropped:473981


,Pool,Make,CommercialName,CategoryOf,MassRunningOrder,Co2EmissionsWltp,BaseWheel,FuelType,EnginePower,ElectricConsumption,InnovativeTechnology,InnovativeEmissionsReductionWltp,ElectricRange,Co2Grade
0,VW-SAIC,SEAT,LEON,M1,1448.000,118.000,2669.000,DIESEL,110.000,0.000,1,1.700,0.000,B
1,VW-SAIC,VOLKSWAGEN,T-ROC,M1,1294.000,137.000,2590.000,PETROL,81.000,0.000,1,0.000,0.000,C
2,FORD,FORD,KUGA,M1,1687.000,135.000,2710.000,DIESEL,110.000,0.000,0,0.000,0.000,C
3,VW-SAIC,VOLKSWAGEN,T-ROC,M1,1294.000,137.000,2590.000,PETROL,81.000,0.000,1,0.000,0.000,C
4,STELLANTIS,OPEL,GRANDLAND,M1,1395.000,140.000,2675.000,PETROL,96.000,0.000,0,2.100,0.000,C


<class 'pandas.io.formats.style.Styler'>


,Column,Non-Null Count
0,Pool,object
1,Make,object
2,CommercialName,object
3,CategoryOf,object
4,MassRunningOrder,float64
5,Co2EmissionsWltp,float64
6,BaseWheel,float64
7,FuelType,object
8,EnginePower,float64
9,ElectricConsumption,float64


<class 'pandas.io.formats.style.Styler'>


,Info
0,"Index: 9446127 entries, 0 to 9920107"
1,Data columns (total 14 columns):
2,13 Co2Grade category
3,"dtypes: category(1), float64(7), int64(1), object(5)"
4,memory usage: 1018.0+ MB


<class 'pandas.io.formats.style.Styler'>


/home/mln/miniconda3/envs/co2/lib/python3.9/site-packages/pandas/io/formats/style.py:4052: RuntimeWarning: invalid value encountered in scalar divide
  end = (x - left) / (right - left)
/home/mln/miniconda3/envs/co2/lib/python3.9/site-packages/pandas/io/formats/style.py:4052: RuntimeWarning: invalid value encountered in scalar divide
  end = (x - left) / (right - left)


,Colonne,Valeurs manquantes (%)
0,Pool,0.00%
1,Make,0.00%
2,CommercialName,0.00%
3,CategoryOf,0.00%
4,MassRunningOrder,0.00%
5,Co2EmissionsWltp,0.00%
6,BaseWheel,0.00%
7,FuelType,0.00%
8,EnginePower,0.00%
9,ElectricConsumption,0.00%


<class 'pandas.io.formats.style.Styler'>


df['Count'] = df.groupby(list(df.columns)).transform('count')
display(df)

df_grouped = df.groupby(['CommercialName', 'FuelType', 'CategoryOf', 'InnovativeTechnology'])\
    .agg({
            'CommercialName': 'count',
            'Co2EmissionsWltp': 'mean',
            'EnginePower': 'mean',
            'ElectricConsumption': 'mean',
            #'InnovativeTechnology': 'mean',
            'InnovativeEmissionsReductionWltp': 'mean',
            'ElectricRange': 'mean',
            'MassRunningOrder': 'mean',
            'BaseWheel': 'mean',
            }).reset_index()

df_grouped['Share'] = df_grouped['Count'] / df_grouped['Count'].sum()
df_grouped.sort_values(by='Share', ascending=False)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9446127 entries, 0 to 9920107
Data columns (total 14 columns):
 #   Column                            Dtype   
---  ------                            -----   
 0   Pool                              object  
 1   Make                              object  
 2   CommercialName                    object  
 3   CategoryOf                        object  
 4   MassRunningOrder                  float64 
 5   Co2EmissionsWltp                  float64 
 6   BaseWheel                         float64 
 7   FuelType                          object  
 8   EnginePower                       float64 
 9   ElectricConsumption               float64 
 10  InnovativeTechnology              int64   
 11  InnovativeEmissionsReductionWltp  float64 
 12  ElectricRange                     float64 
 13  Co2Grade                          category
dtypes: category(1), float64(7), int64(1), object(5)
memory usage: 1018.0+ MB


In [4]:
# Group by the specified columns and compute the mean for the other columns
df = df.groupby(['CommercialName', 'FuelType', 'CategoryOf', 'InnovativeTechnology']).agg({
    'Pool': lambda x: x.mode()[0],
    'Co2EmissionsWltp': 'mean',
    'EnginePower': 'mean',
    'ElectricConsumption': 'mean',
    'InnovativeEmissionsReductionWltp': 'mean',
    'ElectricRange': 'mean',
    'MassRunningOrder': 'mean',
    'BaseWheel': 'mean'
}).reset_index()

# Add a 'Count' column that counts the number of rows in each group
df['Count'] = df.groupby(['CommercialName', 'FuelType', 'CategoryOf', 'InnovativeTechnology']).size().values
df['Share'] = df['Count'] / df['Count'].sum()
df.sort_values(by=['Count'], ascending=False)

,CommercialName,FuelType,CategoryOf,InnovativeTechnology,Pool,Co2EmissionsWltp,EnginePower,ElectricConsumption,InnovativeEmissionsReductionWltp,ElectricRange,MassRunningOrder,BaseWheel,Count,Share
9415,taycan 4s,ELECTRIC,M1,0,VW-SAIC,0.000000,360.000000,0.0,0.000000,276.0,2303.333333,2901.333333,1,0.000106
0,---,PETROL,M1,0,STELLANTIS,132.000000,96.000000,0.0,0.000000,0.0,1323.000000,2605.000000,1,0.000106
1,108,PETROL,M1,0,STELLANTIS,110.441662,53.000000,0.0,0.000000,0.0,921.120371,2340.000000,1,0.000106
2,116 / / D,DIESEL,M1,0,BMW,120.000000,85.000000,0.0,0.000000,0.0,1450.000000,2670.000000,1,0.000106
3,116 / / D,DIESEL,M1,1,BMW,119.500000,85.000000,0.0,1.725000,0.0,1450.000000,2670.000000,1,0.000106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,116D 5-TÜRER F40 B37,DIESEL,M1,0,BMW,121.833333,85.000000,0.0,0.000000,0.0,1454.444444,2670.000000,1,0.000106
12,116D 5-TÜRER F40 B37,DIESEL,M1,1,BMW,120.924051,85.000000,0.0,1.817300,0.0,1454.472574,2670.000000,1,0.000106
13,116I,PETROL,M1,0,BMW,133.725146,80.000000,0.0,1.678363,0.0,1410.614035,2670.000000,1,0.000106
14,116I,PETROL,M1,1,BMW,132.952381,80.373719,0.0,1.887432,0.0,1403.714587,2670.000000,1,0.000106



# Inspection des features
co2.styles.display_info(df, title='DONNEES NETTOYEES', save=True) 
co2.styles.display_describe(df, title='STATISTIQUES DESCRIPTIVES', save=True)

co2.viz.plot_correlation_heatmap(df, interactive=False, save=True)
co2.viz.plot_feature_distributions(df, interactive=False, save=True)
co2.viz.plot_qqplots(df, interactive=False, save=True)

# One-hot encoding des variables catégorielles



# Sauvegarde du jeu de données prêt à l'emploi pour la régression
#co2.data.save_processed_data(df, classification=True, pickle=False)

In [5]:
df.head()

,CommercialName,FuelType,CategoryOf,InnovativeTechnology,Pool,Co2EmissionsWltp,EnginePower,ElectricConsumption,InnovativeEmissionsReductionWltp,ElectricRange,MassRunningOrder,BaseWheel,Count,Share
0,---,PETROL,M1,0,STELLANTIS,132.000000,96.0,0.0,0.000,0.0,1323.000000,2605.0,1,0.000106
1,108,PETROL,M1,0,STELLANTIS,110.441662,53.0,0.0,0.000,0.0,921.120371,2340.0,1,0.000106
2,116 / / D,DIESEL,M1,0,BMW,120.000000,85.0,0.0,0.000,0.0,1450.000000,2670.0,1,0.000106
3,116 / / D,DIESEL,M1,1,BMW,119.500000,85.0,0.0,1.725,0.0,1450.000000,2670.0,1,0.000106
4,116 / / D AUT.,DIESEL,M1,0,BMW,121.000000,85.0,0.0,0.000,0.0,1460.000000,2670.0,1,0.000106


In [6]:
#df = df.drop(columns=['CommercialName'])
df = co2.data.dummify_all_categoricals(df, dummy_columns=['Pool', 'FuelType', 'CategoryOf']
                                       , should_discretize_electricrange=False)

In [7]:
import sys
sys.path.insert(0, '../src/')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import shap
import warnings
import pickle
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_validate, KFold
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

import auto_co2 as co2

%matplotlib inline
warnings.simplefilter(action='ignore', category=FutureWarning)




In [8]:
# Assign the 'Co2EmissionsWltp' column to the variable 'target'
df = co2.data.discretize_co2(df)
target = df['Co2Grade']

# Drop the 'Co2EmissionsWltp' column from the DataFrame 'data'
data = df.drop(columns=['Co2EmissionsWltp', 'Co2Grade', 'CommercialName', 'Share'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=.2, random_state=42)

In [9]:
df.head()

,CommercialName,InnovativeTechnology,Co2EmissionsWltp,EnginePower,ElectricConsumption,InnovativeEmissionsReductionWltp,ElectricRange,MassRunningOrder,BaseWheel,Count,...,FuelType_LPG,FuelType_NATURALGAS,FuelType_NG,FuelType_PETROL,FuelType_PETROL/ELECTRIC,FuelType_UNKNOWN,CategoryOf_M1,CategoryOf_M1G,CategoryOf_m1,Co2Grade
0,---,0,132.000000,96.0,0.0,0.000,0.0,1323.000000,2605.0,1,...,0,0,0,1,0,0,1,0,0,C
1,108,0,110.441662,53.0,0.0,0.000,0.0,921.120371,2340.0,1,...,0,0,0,1,0,0,1,0,0,B
2,116 / / D,0,120.000000,85.0,0.0,0.000,0.0,1450.000000,2670.0,1,...,0,0,0,0,0,0,1,0,0,B
3,116 / / D,1,119.500000,85.0,0.0,1.725,0.0,1450.000000,2670.0,1,...,0,0,0,0,0,0,1,0,0,B
4,116 / / D AUT.,0,121.000000,85.0,0.0,0.000,0.0,1460.000000,2670.0,1,...,0,0,0,0,0,0,1,0,0,C


In [10]:
%%time
dtc = DecisionTreeClassifier(criterion='gini', max_depth=20)
dtc.fit(X_train, y_train)
dtc_pred = dtc.predict(X_test)

co2.viz.plot_confusion_matrix(y_test, dtc_pred, classes=dtc.classes_, title=': Decision Tree', save=True, format='png')
co2.styles.display_classification_report(y_test, dtc_pred)


,precision,recall,f1-score,support
A,0.924,0.950,0.937,242.000
B,0.707,0.631,0.667,130.000
C,0.770,0.794,0.782,417.000
D,0.726,0.726,0.726,369.000
E,0.842,0.834,0.838,397.000
F,0.845,0.824,0.835,205.000
G,0.898,0.919,0.908,124.000
accuracy,0.809,0.809,0.809,0.809
macro avg,0.816,0.811,0.813,1884.000
weighted avg,0.809,0.809,0.809,1884.000


<class 'pandas.io.formats.style.Styler'>
CPU times: user 103 ms, sys: 19.7 ms, total: 123 ms
Wall time: 468 ms


In [11]:
rfc = RandomForestClassifier(n_estimators=200, criterion='entropy', n_jobs=-1, random_state=42)
rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_test)

co2.viz.plot_confusion_matrix(y_test, rfc_pred, palette='Greens', classes=dtc.classes_, title=': Random Forest', save=True, format='png')
co2.styles.display_classification_report(y_test, rfc_pred)

,precision,recall,f1-score,support
A,0.944,0.971,0.957,242.000
B,0.800,0.677,0.733,130.000
C,0.801,0.859,0.829,417.000
D,0.805,0.794,0.799,369.000
E,0.895,0.877,0.885,397.000
F,0.894,0.863,0.878,205.000
G,0.913,0.935,0.924,124.000
accuracy,0.857,0.857,0.857,0.857
macro avg,0.865,0.854,0.858,1884.000
weighted avg,0.857,0.857,0.857,1884.000


<class 'pandas.io.formats.style.Styler'>


In [12]:
encoder = LabelEncoder()
target_enc = encoder.fit_transform(target)

X_x, X_valid_x, y_x, y_valid_x = train_test_split(data, target_enc, test_size=.1)
X_train_x, X_test_x, y_train_x, y_test_x = train_test_split(X_x, y_x, test_size=.2, random_state=42)

xgb_clf = xgb.XGBClassifier(objective='multi:softmax', 
                            n_estimators=600, #400
                            num_class=7, 
                            early_stopping_rounds=20, 
                            eval_metric=['merror','mlogloss'], 
                            seed=42,
                            n_jobs=-1,
                            verbosity=2)
xgb_clf.fit(X_train_x, y_train_x, eval_set=[(X_train_x, y_train_x), (X_test_x, y_test_x)])

[0]	validation_0-merror:0.26479	validation_0-mlogloss:1.46744	validation_1-merror:0.29263	validation_1-mlogloss:1.48441
[1]	validation_0-merror:0.24871	validation_0-mlogloss:1.22332	validation_1-merror:0.27257	validation_1-mlogloss:1.25145
[2]	validation_0-merror:0.23499	validation_0-mlogloss:1.06155	validation_1-merror:0.26018	validation_1-mlogloss:1.09956
[3]	validation_0-merror:0.22570	validation_0-mlogloss:0.94597	validation_1-merror:0.25546	validation_1-mlogloss:0.99001
[4]	validation_0-merror:0.22024	validation_0-mlogloss:0.85829	validation_1-merror:0.25310	validation_1-mlogloss:0.90768
[5]	validation_0-merror:0.21567	validation_0-mlogloss:0.79147	validation_1-merror:0.25251	validation_1-mlogloss:0.84720
[6]	validation_0-merror:0.20490	validation_0-mlogloss:0.73050	validation_1-merror:0.23422	validation_1-mlogloss:0.79163
[7]	validation_0-merror:0.19900	validation_0-mlogloss:0.68911	validation_1-merror:0.23186	validation_1-mlogloss:0.75518
[8]	validation_0-merror:0.19133	validati

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=20,
              enable_categorical=False, eval_metric=['merror', 'mlogloss'],
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=600,
              n_jobs=-1, num_class=7, num_parallel_tree=None, ...)

In [13]:
co2.viz.plot_xgboost(results=xgb_clf.evals_result_, metric='merror', title=' Classifier: Erreur Moyenne')
co2.viz.plot_xgboost(results=xgb_clf.evals_result_, metric='mlogloss', title='Classifier: Moyenne LogLoss')
xgb_pred = xgb_clf.predict(X_test_x)

co2.viz.plot_confusion_matrix(y_test_x, xgb_pred, classes=xgb_clf.classes_, palette='Reds', 
                              title=': XGBoost Classifier', save=True, format='png')
co2.styles.display_classification_report(y_test_x, xgb_pred)

co2.data.save_model(xgb_clf, model_type='xgb')

,precision,recall,f1-score,support
0,0.944,0.969,0.957,227.000
1,0.824,0.680,0.745,103.000
2,0.814,0.841,0.827,384.000
3,0.776,0.749,0.763,343.000
4,0.814,0.866,0.839,344.000
5,0.889,0.856,0.872,187.000
6,0.942,0.907,0.924,107.000
accuracy,0.841,0.841,0.841,0.841
macro avg,0.858,0.838,0.847,1695.000
weighted avg,0.841,0.841,0.840,1695.000


<class 'pandas.io.formats.style.Styler'>
Model saved at ../models/XGBClassifier_20240115_185750.model


/home/mln/miniconda3/envs/co2/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning:

[18:57:50] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1703076482591/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.



In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9416 entries, 0 to 9415
Data columns (total 34 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   InnovativeTechnology              9416 non-null   int64  
 1   EnginePower                       9416 non-null   float64
 2   ElectricConsumption               9416 non-null   float64
 3   InnovativeEmissionsReductionWltp  9416 non-null   float64
 4   ElectricRange                     9416 non-null   float64
 5   MassRunningOrder                  9416 non-null   float64
 6   BaseWheel                         9416 non-null   float64
 7   Count                             9416 non-null   int64  
 8   Pool_BMW                          9416 non-null   int64  
 9   Pool_FORD                         9416 non-null   int64  
 10  Pool_HONDA-GROUP                  9416 non-null   int64  
 11  Pool_HYUNDAI                      9416 non-null   int64  
 12  Pool_K